## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [140]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


### Tasks:
1)	Analyze the results of the experiment and identify the effect of the Treatment on product purchase and Net Incremental Revenue

2)	Build a model to select the best customers to target that maximizes the Incremental Response Rate and Net Incremental Revenue. 

### Exploratory Data Analysis

1. Have basic understanding in promotion proportion, purchase
2. Discover relationship (correlation etc) between V1-V7
3. Decide whether we should scale the features

In [40]:
df=train_data
df.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000
mean,62970.972413,0.012303,1.500662,29.973600,0.000190,1.679608,2.327643,2.502898,1.701694
std,36418.440539,0.110234,0.868234,5.010626,1.000485,0.466630,0.841167,1.117349,0.457517
min,1.000000,0.000000,0.000000,7.104007,-1.684550,1.000000,1.000000,1.000000,1.000000
25%,31467.250000,0.000000,1.000000,26.591501,-0.905350,1.000000,2.000000,2.000000,1.000000
50%,62827.500000,0.000000,2.000000,29.979744,-0.039572,2.000000,2.000000,3.000000,2.000000
75%,94438.750000,0.000000,2.000000,33.344593,0.826206,2.000000,3.000000,4.000000,2.000000
max,126184.000000,1.000000,3.000000,50.375913,1.691984,2.000000,4.000000,4.000000,2.000000


In [41]:
print('We have {} customers, with {} as experiment group and {} as control group.'.format
      (len(df),
       len(df.loc[df.Promotion == 'Yes']),
       len(df.loc[df.Promotion == 'No'])))

We have 84534 customers, with 42364 as experiment group and 42170 as control group


In [51]:
promotion_perc=df.groupby("Promotion")['purchase'].agg(lambda x:round(x.sum()/x.count() *100,2))
promotion_perc

Promotion
No     0.76
Yes    1.70
Name: purchase, dtype: float64

In [61]:
promotion_count=df.groupby("Promotion")['purchase'].agg(lambda x:x.sum())
promotion_count

Promotion
No     319
Yes    721
Name: purchase, dtype: int64

### Deal with Missing Value / Duplicated value

In [43]:
# number of missing value, missing rate, sort
def missing_table(df):
    missing = df.isnull().sum().reset_index().rename(columns = {0:"missing_num"})

    missing['missing_rate']=missing['missing_num']/df.shape[0]

    missing_analy= missing[missing.missing_rate>0].sort_values(by='missing_rate',ascending= False)
    
    if missing_analy.shape[0]==0:
        print("No missing value spot")
    
    return missing_analy

missing_df=missing_table(df)
missing_df

No missing value


,index,missing_num,missing_rate


In [53]:
df.duplicated().sum()

0

### A/B Testing

In [114]:
control_prop= promotion_perc[0]
experiment_prop =promotion_perc[1]

# Formula

In [115]:
# identify treatment effect

experiment_n- control_n


0.93999999999999995

In [ ]:
from scipy.stats import norm


mu_B = 62
mu_A = 60

std_B = 45
std_A = 40

n_B = 4000
n_A = 6000

Z = (mu_experiment - mu_control)/np.sqrt(std_control**2/n_control + std_experiment**2/n_experiment)

pvalue = norm.sf(Z)

print("Z-score: {0}\np-value: {1}".format(Z,p_value))

In [110]:
def find_metrics(df):
    """
    find Incremental Response Rate (IRR) and Net Incremental Revenue(NIR) 
    """
    purchase_treat=df.loc[df.Promotion=='Yes','purchase'].sum()
    purchase_control=df.loc[df.Promotion=='No','purchase'].sum()

    num_treat= (df.Promotion=='Yes').sum()
    num_control=(df.Promotion=='No').sum()
    

    irr= round((purchase_treat/num_treat - purchase_control/num_control)*100,2)
    
    
    revenue,cost=10,0.15
    NIR = round(purchase_treat*revenue-cost*num_treat - purchase_control*revenue,2)
    return irr,NIR

 

In [113]:
#1)	Analyze the results of the experiment
#2) identify the effect of the Treatment on product purchase and Net Incremental Revenue

irr,NIR=find_metrics(df)
print("Incremental Response Rate (IRR) is: {}%, \nNet Incremental Revenue(NIR) is: ${}".format(irr,NIR))

Incremental Response Rate (IRR) is: 0.95%, 
Net Incremental Revenue(NIR) is: $-2334.6


In [134]:
from tqdm import tqdm

# Randomdized Experiment
def permtest(df, n_trials = 10000):
    """
    Compute a confidence interval for the irr and nir using a permutation method. 
    
    Input parameters:
        df: original dataframe on which to perform the test
        alternative: type of test to perform, {'less', 'greater'}
        n_trials: number of permutation trials to perform
    
    Output value:
        a confidence interval containing 95% of values
    """
    # initialize storage of permuted irr and nir
    sample_irr = []
    sample_nir = []
    
    # For each trial...
    for _ in tqdm(range(n_trials)):
        
        # randomly permute the grouping labels
        df.Promotion = np.random.permutation(df.Promotion)
        
        # compute the irr and nir on the permuted dataset
        irr ,nir = find_metrics(df)

        # and add the value to the list of samples
        sample_irr.append(irr)
        sample_nir.append(nir)
    
    irr_ci = []
    irr_ci.append(np.percentile(np.array(sample_irr),0.025))
    irr_ci.append(np.percentile(np.array(sample_irr),0.975))
    nir_ci = []
    nir_ci.append(np.percentile(np.array(sample_nir),0.025))
    nir_ci.append(np.percentile(np.array(sample_nir),0.975))

    return irr_ci, nir_ci

In [133]:
# irr_ci, nir_ci = permtest(train_data, n_trials = 1000)
# irr_ci, nir_ci

print("{},{}".format(round(irr_ci,3),round(nir_ci,3)))

100%|██████████| 1000/1000 [00:39<00:00, 25.27it/s]


([-0.20750249999999998, -0.16], [-7204.6099999999997, -6999.7950000000001])

## Part 2: Find promotion strategy

1. Machine Learning prediction
     * Random Forest
     * Xgboost

In [126]:
def compare_features(row):
    '''
    A helper function to get the predicted probability of purchasing for a given individual and make a decision based on a
    chosen boundary.
    
    INPUTS:
    row: an array containing the values of interest in a row
    
    OUTPUT:
    a string specifying whether a given individual (characterized by a row) should be sent the promotion or not
    '''
    if tree.predict_proba(row.values.reshape(1,-1))[0][1] > 0.4:
        return 'Yes'
    else:
        return 'No'

In [125]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''

    promotion = df.apply(compare_features, axis = 1)
    return promotion

### Feature Engineering

In [138]:
import os
os.listdir()

['.Trash-0',
 'Test.csv',
 'training.csv',
 'Starbucks-zh.ipynb',
 '.ipynb_checkpoints',
 '__pycache__',
 'test_results.py',
 'Starbucks.ipynb']

In [145]:
train_data = pd.read_csv('./training.csv')
test_data = pd.read_csv('./Test.csv')
from sklearn import model_selection

# split data into train and valid
train, valid = model_selection.train_test_split(train_data, test_size=0.2,random_state=1)

In [ ]:
Y_train = train['response']
X_train = train[features]

Y_valid = valid['response']
X_valid = valid[features]

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)


In [159]:
#!pip install numpy==1.8 Make sure Numpy version is 1.8

from imblearn.over_sampling import SMOTE
# up sample only the train dataset with SMOTE
sm = SMOTE(random_state=1, ratio = 1.0)
X_train_upsamp, Y_train_upsamp = sm.fit_sample(X_train, Y_train)
X_train_upsamp = pd.DataFrame(X_train_upsamp, columns=features)

Y_train_upsamp = pd.Series(Y_train_upsamp)

ImportError: numpy.core.multiarray failed to import

In [ ]:

# generate features and labels
Y_train = train['response']
X_train = train[features]

Y_valid = valid['response']
X_valid = valid[features]


### Random Forest

### Xgboost

In [122]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

print("Default Strategy")

test_results(promotion_strategy)

Default Strategy
Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0096.

Your nir with this strategy is -1132.20.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.0095931582782501079, -1132.1999999999998)